In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define the main data directory
# This is where Kaggle stores the dataset you linked
base_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/'

# Define paths for training, validation, and test sets
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Define paths for NORMAL and PNEUMONIA images within the training set
train_normal_dir = os.path.join(train_dir, 'NORMAL')
train_pneumonia_dir = os.path.join(train_dir, 'PNEUMONIA')

# Get the number of images in each category
num_normal_train = len(os.listdir(train_normal_dir))
num_pneumonia_train = len(os.listdir(train_pneumonia_dir))

print(f"Total training images (NORMAL): {num_normal_train}")
print(f"Total training images (PNEUMONIA): {num_pneumonia_train}")
print("---")
print(f"Total training images: {num_normal_train + num_pneumonia_train}")


Total training images (NORMAL): 1341
Total training images (PNEUMONIA): 3875
---
Total training images: 5216


In [ ]:
# Let's visualize some of the images
import cv2 # A library for image processing

# Get a few sample image file names
normal_images = os.listdir(train_normal_dir)[:5]
pneumonia_images = os.listdir(train_pneumonia_dir)[:5]

# Create a figure to display the images
fig = plt.figure(figsize=(10, 5))

# Plot the NORMAL images
for i, img_name in enumerate(normal_images):
    img_path = os.path.join(train_normal_dir, img_name)
    img = cv2.imread(img_path)
    
    ax = fig.add_subplot(2, 5, i + 1)
    ax.imshow(img)
    ax.set_title('NORMAL')
    ax.axis('off')

# Plot the PNEUMONIA images
for i, img_name in enumerate(pneumonia_images):
    img_path = os.path.join(train_pneumonia_dir, img_name)
    img = cv2.imread(img_path)
    
    ax = fig.add_subplot(2, 5, i + 6)
    ax.imshow(img)
    ax.set_title('PNEUMONIA')
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Import the necessary tool from TensorFlow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Create a data generator for the training set
# We will also apply 'data augmentation' here to make our model more robust.
# Data augmentation creates modified versions of our images (rotated, zoomed, etc.)
train_datagen = ImageDataGenerator(
    rescale=1./255,      # Normalize pixel values to be between 0 and 1
    rotation_range=20,   # Randomly rotate images
    width_shift_range=0.1, # Randomly shift images horizontally
    height_shift_range=0.1,# Randomly shift images vertically
    shear_range=0.1,     # Apply shear transformation
    zoom_range=0.1,      # Randomly zoom in on images
    horizontal_flip=True,# Randomly flip images horizontally
    fill_mode='nearest'  # How to fill in newly created pixels
)

# 2. Create a data generator for the validation and test sets
# We only need to normalize these images, no augmentation needed.
test_datagen = ImageDataGenerator(rescale=1./255)

# 3. Use the generators to load images from their directories
IMG_SIZE = (150, 150) # Define a standard size for all images
BATCH_SIZE = 32       # How many images to process at a time

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary' # Since we have two classes (NORMAL, PNEUMONIA)
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Print out the results
print("\nData generators created successfully!")
print("Keras has automatically assigned numerical labels to our classes:")
print(train_generator.class_indices)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

# 1. Load the VGG16 model, pre-trained on ImageNet data
# We don't include the top (classification) layer because we'll create our own
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

# 2. Freeze the convolutional base
# This prevents the weights in VGG16 from being updated during training
conv_base.trainable = False

# 3. Create our new model on top of the VGG16 base
model = models.Sequential()
model.add(conv_base) # Add the frozen VGG16 base
model.add(layers.Flatten()) # Flatten the output to a 1D vector
model.add(layers.Dense(256, activation='relu')) # Add a new dense layer for our specific task
model.add(layers.Dense(1, activation='sigmoid')) # The final output layer (1 neuron for binary classification)

# 4. Compile the model
# This configures the model for training
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=2e-5),
              metrics=['accuracy'])

# 5. Print a summary of our model architecture
model.summary()


In [ ]:
# Train the model using the data generators
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # Number of batches to draw from the generator per epoch
    epochs=10,            # How many times to go through the entire dataset
    validation_data=validation_generator,
    validation_steps=50   # Number of batches to draw from the validation generator
)

# Save the trained model
model.save('pneumonia_classifier_v1.keras')

print("\nTraining complete and model saved!")


In [ ]:
# Retrieve accuracy and loss from the history object
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(acc) + 1)

# Create a figure with two subplots
plt.figure(figsize=(12, 5))

# Plot training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, 'bo-', label='Training Accuracy')
plt.plot(epochs_range, val_acc, 'ro-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, 'bo-', label='Training Loss')
plt.plot(epochs_range, val_loss, 'ro-', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# We need to build a new model that includes Dropout layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

# Load the base model again
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

# Freeze the base
conv_base.trainable = False

# Create the new model with Dropout layers
mc_model = models.Sequential()
mc_model.add(conv_base)
mc_model.add(layers.Flatten())
mc_model.add(layers.Dense(256, activation='relu'))
mc_model.add(layers.Dropout(0.5)) # <--- ADDED DROPOUT
mc_model.add(layers.Dense(1, activation='sigmoid'))

# Compile the new model
mc_model.compile(loss='binary_crossentropy',
                 optimizer=optimizers.RMSprop(learning_rate=2e-5),
                 metrics=['accuracy'])

# Print the summary of the new model
mc_model.summary()


In [ ]:
# Train the new model with dropout
history_mc = mc_model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,  # We'll train for the same number of epochs for comparison
    validation_data=validation_generator,
    validation_steps=50
)

# Save the new trained model
mc_model.save('pneumonia_classifier_mc_v1.keras')

print("\nTraining of MC model complete and model saved!")


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# This is the core function for Monte Carlo predictions
def mc_predict(model, img_path, num_samples=100):
    """
    Performs Monte Carlo predictions by running the model multiple times with dropout enabled.
    """
    # 1. Load and preprocess the image
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.  # Normalize the image

    # 2. Create a list to store predictions
    predictions = []

    # 3. Run prediction N times
    for _ in range(num_samples):
        # The 'training=True' flag is the key to activating dropout during prediction
        pred = model(img_tensor, training=True)
        predictions.append(pred.numpy().flatten()[0])

    # 4. Calculate mean and standard deviation
    mean_prediction = np.mean(predictions)
    std_dev = np.std(predictions)

    return mean_prediction, std_dev

# --- Let's test it on one image ---

# Get a sample image from the validation set (which the model hasn't trained on)
# You can try both a NORMAL and a PNEUMONIA image to see the difference
sample_normal_image_path = os.path.join(validation_dir, 'NORMAL', os.listdir(os.path.join(validation_dir, 'NORMAL'))[0])
sample_pneumonia_image_path = os.path.join(validation_dir, 'PNEUMONIA', os.listdir(os.path.join(validation_dir, 'PNEUMONIA'))[0])

# Perform MC prediction on the NORMAL image
mean_norm, std_norm = mc_predict(mc_model, sample_normal_image_path)
print(f"--- Prediction for a NORMAL image ---")
print(f"Predicted Probability (of Pneumonia): {mean_norm:.4f}")
print(f"Uncertainty (Standard Deviation): {std_norm:.4f}")
print("\n")

# Perform MC prediction on the PNEUMONIA image
mean_pneu, std_pneu = mc_predict(mc_model, sample_pneumonia_image_path)
print(f"--- Prediction for a PNEUMONIA image ---")
print(f"Predicted Probability (of Pneumonia): {mean_pneu:.4f}")
print(f"Uncertainty (Standard Deviation): {std_pneu:.4f}")



In [ ]:
# This function is similar to the one before, but it returns the raw predictions
def get_mc_predictions(model, img_path, num_samples=100):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.
    
    predictions = []
    for _ in range(num_samples):
        pred = model(img_tensor, training=True)
        predictions.append(pred.numpy().flatten()[0])
        
    return predictions

# Get the raw predictions for both images
normal_preds = get_mc_predictions(mc_model, sample_normal_image_path)
pneumonia_preds = get_mc_predictions(mc_model, sample_pneumonia_image_path)

# Plot the distributions
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(normal_preds, bins=20, alpha=0.7, label='Normal Image Predictions')
plt.title('Prediction Distribution (High Uncertainty)')
plt.xlabel('Predicted Probability of Pneumonia')
plt.ylabel('Frequency')
plt.xlim(0, 1) # Keep x-axis consistent
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(pneumonia_preds, bins=10, alpha=0.7, color='r', label='Pneumonia Image Predictions')
plt.title('Prediction Distribution (Low Uncertainty)')
plt.xlabel('Predicted Probability of Pneumonia')
plt.xlim(0, 1) # Keep x-axis consistent
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# --- FIX: Re-define the variables for number of images ---
# This ensures the variables are available in the current scope.
train_normal_dir = os.path.join(train_dir, 'NORMAL')
train_pneumonia_dir = os.path.join(train_dir, 'PNEUMONIA')
num_normal_train = len(os.listdir(train_normal_dir))
num_pneumonia_train = len(os.listdir(train_pneumonia_dir))
# --- End of FIX ---


# Step 1: Calculate Class Weights to handle data imbalance
# Formula: weight_for_class_X = (total_samples / (num_classes * num_samples_of_class_X))

total_train_samples = num_normal_train + num_pneumonia_train
weight_for_0 = (1 / num_normal_train) * (total_train_samples / 2.0)
weight_for_1 = (1 / num_pneumonia_train) * (total_train_samples / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print(f"Weight for class 0 (NORMAL): {class_weight[0]:.2f}")
print(f"Weight for class 1 (PNEUMONIA): {class_weight[1]:.2f}")
print("\n--- Re-training the MC model with class weights ---\n")

# We need to re-compile the model to reset its state before re-training
mc_model.compile(loss='binary_crossentropy',
                 optimizer=optimizers.RMSprop(learning_rate=2e-5),
                 metrics=['accuracy'])

# Step 2: Re-train the model, but this time passing the class_weight argument
history_weighted = mc_model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50,
    class_weight=class_weight  # <--- THE IMPORTANT ADDITION
)

# Save the new, improved model
mc_model.save('pneumonia_classifier_mc_weighted_v2.keras')
print("\nTraining with weights complete and new model saved!")


In [ ]:
# ==================================================================
# FIX 2: Re-define ALL necessary variables to make the cell independent
# ==================================================================

# --- Re-defining file paths and counts ---
base_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/'
train_dir = os.path.join(base_dir, 'train')
train_normal_dir = os.path.join(train_dir, 'NORMAL')
train_pneumonia_dir = os.path.join(train_dir, 'PNEUMONIA')
num_normal_train = len(os.listdir(train_normal_dir))
num_pneumonia_train = len(os.listdir(train_pneumonia_dir))

# --- Re-building the MC model architecture ---
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))
conv_base.trainable = False

mc_model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# ==================================================================
# Original code starts here
# ==================================================================

# Step 1: Calculate Class Weights
total_train_samples = num_normal_train + num_pneumonia_train
weight_for_0 = (1 / num_normal_train) * (total_train_samples / 2.0)
weight_for_1 = (1 / num_pneumonia_train) * (total_train_samples / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}

print(f"Weight for class 0 (NORMAL): {class_weight[0]:.2f}")
print(f"Weight for class 1 (PNEUMONIA): {class_weight[1]:.2f}")
print("\n--- Re-training the MC model with class weights ---\n")

# Compile the model
mc_model.compile(loss='binary_crossentropy',
                 optimizer=optimizers.RMSprop(learning_rate=2e-5),
                 metrics=['accuracy'])

# Step 2: Re-train the model with class_weight
history_weighted = mc_model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=50,
    class_weight=class_weight
)

# Save the new, improved model
mc_model.save('pneumonia_classifier_mc_weighted_v2.keras')
print("\nTraining with weights complete and new model saved!")



In [ ]:
# Step 1: Create a data generator for the test set.
# It's crucial that shuffle=False, so we can analyze predictions later if needed.
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False) # Do not shuffle the test data

# Step 2: Evaluate the final, weighted model on the test set.
print("\n--- Evaluating the final model on the unseen test set ---")
test_loss, test_accuracy = mc_model.evaluate(test_generator)

print(f"\nFinal Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Final Test Loss: {test_loss:.4f}")

# Step 3: Let's get more detailed metrics
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get the ground truth labels
y_true = test_generator.classes

# Get the model's predictions
# We'll use the standard prediction method here, not MC, for a baseline report.
y_pred_probs = mc_model.predict(test_generator)
y_pred = np.where(y_pred_probs > 0.5, 1, 0).flatten()

# Generate and print the classification report
print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=['NORMAL (Class 0)', 'PNEUMONIA (Class 1)']))

# Generate and print the confusion matrix
print("\n--- Confusion Matrix ---")
print("         Predicted 0   Predicted 1")
print("Actual 0", confusion_matrix(y_true, y_pred)[0])
print("Actual 1", confusion_matrix(y_true, y_pred)[1])


In [ ]:
# Step 1: Install Gradio library
# The '!' tells the notebook to run this as a shell command
!pip install gradio -q

import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Step 2: Load our best trained model
# Make sure the filename is correct
final_model = load_model('pneumonia_classifier_mc_weighted_v2.keras')

# Step 3: Create a single, all-in-one prediction function for Gradio
def predict_and_visualize(input_image):
    """
    Takes a user-uploaded image, performs MC prediction, and returns results and a plot.
    """
    # Gradio provides the image as a numpy array, so we don't need to load it from a path
    # We just need to resize and normalize it
    img_array = np.array(input_image)
    img_resized = np.resize(img_array, (150, 150, 3)) # Resize to our model's input size
    img_tensor = np.expand_dims(img_resized, axis=0)
    img_tensor = img_tensor / 255.0 # Normalize

    # Perform Monte Carlo predictions
    num_samples = 100
    predictions = []
    for _ in range(num_samples):
        pred = final_model(img_tensor, training=True)
        predictions.append(pred.numpy().flatten()[0])

    # Calculate results
    mean_prediction = np.mean(predictions)
    std_dev = np.std(predictions)
    confidence = (1 - std_dev) * 100

    # Determine the final label
    if mean_prediction > 0.5:
        label = "PNEUMONIA"
    else:
        label = "NORMAL"

    # Create the histogram plot
    fig = plt.figure()
    plt.hist(predictions, bins=20, alpha=0.7)
    plt.title(f"Prediction Distribution (Uncertainty: {std_dev:.3f})")
    plt.xlabel("Predicted Probability of Pneumonia")
    plt.ylabel("Frequency")
    plt.xlim(0, 1)
    # We can't show the plot directly, but we return the figure object
    
    # Format the results for display
    results = f"Final Prediction: {label}\n"
    results += f"Confidence Score: {confidence:.2f}%\n"
    results += f"(Raw Pneumonia Probability: {mean_prediction:.3f})"

    return results, fig

print("Gradio installed and prediction function is ready!")



In [ ]:
# Launch the Gradio interface
gr.Interface(fn=predict_and_visualize,
             inputs=gr.Image(type="pil"),
             outputs=["text", "plot"],
             title="Pneumonia Detection with Uncertainty Estimation",
             description="Upload a chest X-ray image to get a prediction. The model indicates its confidence through an uncertainty score and a prediction distribution plot.").launch(debug=True)


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# We will create ONE generator and use it to split the MAIN training data
# This ignores the tiny 'val' folder and creates a proper, larger validation set.

datagen_with_split = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # <--- THE MAGIC PARAMETER: Reserve 20% of data for validation
)

# Now, create two generators from the SAME directory (train_dir)
# but specify which subset to use for each.

IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# The main training folder
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'

print("--- Creating new, improved data generators ---")

# Generator for the new, larger training set (80%)
train_generator_new = datagen_with_split.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify this is the training subset
)

# Generator for the new, larger validation set (20%)
validation_generator_new = datagen_with_split.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation' # Specify this is the validation subset
)

print("\n--- Re-calculating class weights for the new training split ---")
# We need to recalculate weights based on the new 80% training split
# The generator tells us the new counts
num_normal_new = np.sum(train_generator_new.classes == 0)
num_pneumonia_new = np.sum(train_generator_new.classes == 1)
total_new_train = num_normal_new + num_pneumonia_new

weight_for_0_new = (1 / num_normal_new) * (total_new_train / 2.0)
weight_for_1_new = (1 / num_pneumonia_new) * (total_new_train / 2.0)
class_weight_new = {0: weight_for_0_new, 1: weight_for_1_new}

print(f"New training samples: {total_new_train}")
print(f"New validation samples: {validation_generator_new.n}")
print(f"New weight for NORMAL: {class_weight_new[0]:.2f}")
print(f"New weight for PNEUMONIA: {class_weight_new[1]:.2f}")

2025-12-05 17:34:04.352032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764956044.373515    1398 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764956044.379975    1398 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

--- Creating new, improved data generators ---
Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.

--- Re-calculating class weights for the new training split ---
New training samples: 4173
New validation samples: 1043
New weight for NORMAL: 1.94
New weight for PNEUMONIA: 0.67


In [3]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

# --- Re-building the MC model architecture ---
conv_base_new = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(150, 150, 3))
conv_base_new.trainable = False

mc_model_new = models.Sequential([
    conv_base_new,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# --- Compile the new model ---
mc_model_new.compile(loss='binary_crossentropy',
                     optimizer=optimizers.RMSprop(learning_rate=2e-5),
                     metrics=['accuracy'])

print("--- Starting final training with improved data generators ---")

# --- Train the model with the NEW generators and NEW class weights ---
history_final = mc_model_new.fit(
    train_generator_new,
    steps_per_epoch=100,  # We can keep this at 100
    epochs=15,  # Let's train for a bit longer (15 epochs) as we have more data
    validation_data=validation_generator_new,
    validation_steps=50, # We can keep this at 50
    class_weight=class_weight_new
)

# --- Save the final, most powerful model ---
mc_model_new.save('pneumonia_classifier_final_v3.keras')
print("\nFinal training complete! The best model yet has been saved.")


I0000 00:00:1764956122.629303    1398 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1764956122.629883    1398 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


--- Starting final training with improved data generators ---


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1764956126.045772    1466 service.cc:148] XLA service 0x79f478003900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764956126.045807    1466 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1764956126.045811    1466 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1764956126.349740    1466 cuda_dnn.cc:529] Loaded cuDNN version 90300


  2/100 ━━━━━━━━━━━━━━━━━━━━ 9s 95ms/step - accuracy: 0.6016 - loss: 0.8189 

I0000 00:00:1764956132.953988    1466 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 617ms/step - accuracy: 0.6664 - loss: 0.6236 - val_accuracy: 0.8150 - val_loss: 0.4083
Epoch 2/15
  1/100 ━━━━━━━━━━━━━━━━━━━━ 9s 92ms/step - accuracy: 0.8750 - loss: 0.3487

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


100/100 ━━━━━━━━━━━━━━━━━━━━ 24s 238ms/step - accuracy: 0.8667 - loss: 0.3461 - val_accuracy: 0.8715 - val_loss: 0.3409
Epoch 3/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 52s 523ms/step - accuracy: 0.8814 - loss: 0.3235 - val_accuracy: 0.8878 - val_loss: 0.2885
Epoch 4/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 25s 247ms/step - accuracy: 0.8963 - loss: 0.2563 - val_accuracy: 0.8658 - val_loss: 0.2995
Epoch 5/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 53s 535ms/step - accuracy: 0.9016 - loss: 0.2499 - val_accuracy: 0.9070 - val_loss: 0.2284
Epoch 6/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 24s 245ms/step - accuracy: 0.9081 - loss: 0.2398 - val_accuracy: 0.8888 - val_loss: 0.2555
Epoch 7/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 53s 528ms/step - accuracy: 0.9121 - loss: 0.2308 - val_accuracy: 0.9003 - val_loss: 0.2340
Epoch 8/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 25s 251ms/step - accuracy: 0.8952 - loss: 0.2458 - val_accuracy: 0.8907 - val_loss: 0.2608
Epoch 9/15
100/100 ━━━━━━━━━━━━━━━━━━━━ 53s 533ms/step - accuracy: 0.9217 - loss: 0.2054 - val

In [ ]:
# Step 1: Create a data generator for the test set (if not already created in this session)
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
test_datagen = ImageDataGenerator(rescale=1./255) # Just rescale, no augmentation

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False)

# Step 2: Evaluate the FINAL model on the unseen test set
print("\n--- Evaluating the FINAL, IMPROVED model on the unseen test set ---")
# Make sure to use mc_model_new, our latest and greatest model
test_loss, test_accuracy = mc_model_new.evaluate(test_generator)

print(f"\nFinal Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Final Test Loss: {test_loss:.4f}")

# Step 3: Generate detailed metrics
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = test_generator.classes
y_pred_probs = mc_model_new.predict(test_generator)
y_pred = np.where(y_pred_probs > 0.5, 1, 0).flatten()

print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=['NORMAL (Class 0)', 'PNEUMONIA (Class 1)']))

print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_true, y_pred))


In [4]:
# Make sure all necessary libraries are imported for this new session
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from PIL import Image

print("--- Preparing the Final Demo ---")

# Step 1: Load our BEST and FINAL trained model
try:
    final_model = load_model('pneumonia_classifier_final_v3.keras')
    print("Model 'pneumonia_classifier_final_v3.keras' loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please ensure you have run the final training cell to generate the model file.")

# Step 2: Create the final, all-in-one prediction function
def predict_final(input_image):
    """
    Takes a user-uploaded image, performs MC prediction with the FINAL model,
    and returns polished results and a plot.
    """
    if input_image is None:
        return "Please upload an image.", None

    # Gradio provides a PIL image, convert to numpy array and resize
    img = np.array(input_image.resize((150, 150)))
    
    # Ensure image is 3-channel (RGB)
    if img.ndim == 2: # If it's grayscale
        img = np.stack((img,)*3, axis=-1)
    
    img_tensor = np.expand_dims(img, axis=0)
    img_tensor = img_tensor / 255.0  # Normalize

    # --- Perform Monte Carlo predictions ---
    num_samples = 100
    predictions = []
    for _ in range(num_samples):
        pred = final_model(img_tensor, training=True) # Enable dropout for uncertainty
        predictions.append(pred.numpy().flatten()[0])

    # --- Calculate and interpret results ---
    mean_prediction = np.mean(predictions)
    std_dev = np.std(predictions)
    
    # We define confidence differently now: as certainty in the chosen class
    if mean_prediction > 0.5:
        label = "PNEUMONIA"
        confidence = mean_prediction * 100
    else:
        label = "NORMAL"
        confidence = (1 - mean_prediction) * 100
    
    uncertainty_score = std_dev * 100 # As a percentage

    # --- Create the histogram plot ---
    fig = plt.figure(figsize=(8, 5))
    plt.hist(predictions, bins=20, alpha=0.8, color='#007BFF')
    plt.title("Prediction Distribution", fontsize=16)
    plt.xlabel("Model's Prediction (Probability of Pneumonia)", fontsize=12)
    plt.ylabel("Frequency (out of 100 runs)", fontsize=12)
    plt.xlim(0, 1)
    plt.axvline(x=mean_prediction, color='r', linestyle='--', label=f'Average: {mean_prediction:.2f}')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # --- Format the final results for display ---
    results = {
        "Final Prediction": label,
        f"Confidence in '{label}'": f"{confidence:.2f}%",
        "Uncertainty Score": f"{uncertainty_score:.2f}% (Lower is better)"
    }

    return results, fig

print("Final prediction function is ready!")


--- Preparing the Final Demo ---
Model 'pneumonia_classifier_final_v3.keras' loaded successfully!
Final prediction function is ready!


In [ ]:
# --- Launch the final, professional Gradio interface (FIXED VERSION) ---

# FIX 2: Define the exact path where the examples are located
example_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/test/'

demo = gr.Interface(
    fn=predict_final,
    inputs=gr.Image(type="pil", label="Upload Chest X-ray Image"),
    outputs=[
        # FIX 1: Change the output type to a simple Textbox to match our function's output
        gr.Textbox(label="Analysis Results"),
        gr.Plot(label="Prediction Distribution & Uncertainty")
    ],
    title="🩺 AI-Powered Pneumonia Detection with Uncertainty Estimation",
    description="""
    **Welcome! This is a proof-of-concept tool, not for medical use.**
    Upload a chest X-ray to see the model's prediction. The system uses a deep learning model (VGG16) and Monte Carlo Dropout to not only classify the image but also to estimate its own uncertainty.
    - **High Confidence & Low Uncertainty:** The model is sure of its decision.
    - **Low Confidence & High Uncertainty:** The model is unsure and suggests the case might be complex or unusual, requiring expert review.
    """,
    article="**Developed by:** [Your Name Here] - **Model Version:** v3.0",
    examples=[
        [example_path + 'NORMAL/NORMAL2-IM-1427-0001.jpeg'],
        [example_path + 'PNEUMONIA/person100_bacteria_475.jpeg']
    ],
    allow_flagging='never'
)

# FIX 2 (continued): Add the allowed_paths parameter to the launch() function
demo.launch(debug=True, allowed_paths=[example_path])


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://54dde5ec503767e295.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2231, in process_api
    inputs = await self.preprocess_data(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1877, in preprocess_data
    inputs_cached = await processing_utils.async_move_files_to_cache(
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/processing_utils.py", line 676, in async_move_files_to_cache
    return await client_utils.async_tr